# Clase 11

In [1]:
#Frase
sentence = 'Chile ganó la Copa América el año 2015 y el año 2016'
#Tokenizar esta cadena con metodo split
token_sequence = sentence.split()
print(token_sequence)

['Chile', 'ganó', 'la', 'Copa', 'América', 'el', 'año', '2015', 'y', 'el', 'año', '2016']


In [2]:
# Otra forma de hacer esto mismo
token_sequence = str.split(sentence)
print(token_sequence)

['Chile', 'ganó', 'la', 'Copa', 'América', 'el', 'año', '2015', 'y', 'el', 'año', '2016']


## One hot Encoding
Cuando ya tenemos tokenizado el texto, 
podemos hacer alguna transformacion 
que nos lleve este texto a algun objeto matematico
El 'One hot Encoding' sirve para mantener la gramática y el orden de las oraciones, pero no se encodifica el significado de las palabras.

In [3]:
import numpy as np
#Identificar nuestro lexico, tokens unicos
vocab = sorted(set(token_sequence))
print(vocab)
print(type(vocab))

['2015', '2016', 'América', 'Chile', 'Copa', 'año', 'el', 'ganó', 'la', 'y']
<class 'list'>


In [4]:
# El numero de tokens en la oracion
num_tokens = len(token_sequence)
print(num_tokens)

#La cantidad de tokens en el léxico
vocab_size = len(vocab)
print(vocab_size)

12
10


Cada palabra de una oración, será un vector de largo 10 con un 1 que indicará que token se quiere representar 

'2015' = $[ 1 0 0 0 0 0 0 0 0 0]$

'ganó' = $[ 0 0 0 0 0 0 0 1 0 0]$

Entonces una oración debe ser una matriz, en la que cada columna representará una palabra

In [5]:
onehot_vectors = np.zeros((num_tokens, vocab_size), int)

for i, word in enumerate(token_sequence):
    onehot_vectors[i, vocab.index(word)] = 1

#Representacion vectorial de una frase
rep_vec = np.transpose(onehot_vectors)
print(rep_vec)
print(rep_vec.shape)

[[0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 1 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 1 0]
 [0 0 0 0 0 1 0 0 0 1 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0]]
(10, 12)


Ahora se utilizará la libreria 'pandas' para visualizar mejor esto

In [6]:
import pandas as pd
df = pd.DataFrame(onehot_vectors, columns=vocab)
df

,2015,2016,América,Chile,Copa,año,el,ganó,la,y
0,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,1,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0
5,0,0,0,0,0,0,1,0,0,0
6,0,0,0,0,0,1,0,0,0,0
7,1,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,1
9,0,0,0,0,0,0,1,0,0,0


In [7]:
df[df == 0] = ''
df

,2015,2016,América,Chile,Copa,año,el,ganó,la,y
0,,,,1,,,,,,
1,,,,,,,,1,,
2,,,,,,,,,1,
3,,,,,1,,,,,
4,,,1,,,,,,,
5,,,,,,,1,,,
6,,,,,,1,,,,
7,1,,,,,,,,,
8,,,,,,,,,,1
9,,,,,,,1,,,


## Bolsa de Palabras binarias
Representar cada frase que tengamos en nuestro corpus, como un vector de largo igual al numero de tokens y donde habrá 1 si la palabra está en la frase, sino 0.


In [8]:
sentences = """Chile ganó la Copa América los años 2015 y 2016. \n"""
sentences += """Chile no fué al mundial.\n"""
sentences += "La Copa América es en Argentina.\n"
sentences += """En el 2015 y 2016 no estaba en Chile."""

In [9]:
#La variable corpus será un diccionario 
corpus = {}
#Separamos las sentences por \n y
#Etiquetamos con un 1 si la palabra está en la oración
for i, sent in enumerate(sentences.split('\n')):
    corpus['sent{}'.format(i)] = dict((tok, 1) for tok in sent.split())

In [18]:
#Llevaremos el corpus a un dataframe
#Colocaremos un 0 si un token no está en la oracion
df = pd.DataFrame.from_records(corpus).fillna(0).astype(int).T
print(df[df.columns[:20]])

       Chile  ganó  la  Copa  América  los  años  2015  y  2016.  no  fué  al  \
sent0      1     1   1     1        1    1     1     1  1      1   0    0   0   
sent1      1     0   0     0        0    0     0     0  0      0   1    1   1   
sent2      0     0   0     1        1    0     0     0  0      0   0    0   0   
sent3      0     0   0     0        0    0     0     1  1      0   1    0   0   

       mundial.  La  es  en  Argentina.  En  el  
sent0         0   0   0   0           0   0   0  
sent1         1   0   0   0           0   0   0  
sent2         0   1   1   1           1   0   0  
sent3         0   0   0   1           0   1   1  


In [19]:
sentences.split('\n')

['Chile ganó la Copa América los años 2015 y 2016. ',
 'Chile no fué al mundial.',
 'La Copa América es en Argentina.',
 'En el 2015 y 2016 no estaba en Chile.']

¿Que ocurre si calculamos el producto punto entre las frases?

$\vec{sent0} \cdot \vec{sent1}$ 

Nos da la proyeccion de una frase sobre la otra y su valor representa cuantas veces tiene una frase las palabras de la otra frase
Si el resultado es $1$, las frases tienen 1 palabra en común, si nos da $2$, las frases tienen 2 palabras en común, etc. 

### Como calcular producto punto con numpy

In [11]:
v1 = np.array([1, 2, 3])
v2 = np.array([2, 3, 4])
v1.dot(v2)

20

In [1]:
## Producto punto entre oraciones
print(df)
print('Producto punto entre frase 0 y frase 1')
print(df.T.sent0.dot(df.T.sent1))

print('Producto punto entre frase 1 y frase 2')
print(df.T.sent1.dot(df.T.sent2))


NameError: name 'df' is not defined

In [13]:
# Tambien podemos ver cuales son las palabras que se comparten
[(k, v) for (k, v) in (df.T.sent0 & df.T.sent3).items() if v]

[('2015', 1), ('y', 1)]

Sin embargo, la tokenizacion no es efectiva, ya que en el ejemplo de abajo podemos ver que esta el token "Chile" y "Chile."
Para resolver esto, utilizaremos la funcion split de la libreria "re"

In [14]:
print(df.T.sent0)

Chile         1
ganó          1
la            1
Copa          1
América       1
los           1
años          1
2015          1
y             1
2016.         1
no            0
fué           0
al            0
mundial.      0
La            0
es            0
en            0
Argentina.    0
En            0
el            0
2016          0
estaba        0
Chile.        0
Name: sent0, dtype: int32


In [15]:
import re
sentence = """Chile ganó \x0b la Copa América \n los años 2015 y 2016."""
tokens = re.split(r'[-\s.,;!?]+', sentence)
print(tokens)

['Chile', 'ganó', 'la', 'Copa', 'América', 'los', 'años', '2015', 'y', '2016', '']
